In [1]:
# Keeping this class here for now (easy to test). Will move this to a separate .py file when we do code-cleanup/ submission

import psycopg2
import psycopg2.extras
import re
import pandas


class FoodViolationData:

    def __init__(self, connection_string):
        self.conn = psycopg2.connect(connection_string)

    class FoodViolationQuery:

        def __init__(self, product_data):
            self.joins = []
            self.conditions = []
            self.parameters = {}
            self.product_data = product_data

            self.named_params = {}

        def __param_index(self, param_name):

            if param_name not in self.named_params:
                self.named_params.update({param_name: 0})
                return 0
            else:
                i = self.named_params[param_name] + 1
                self.named_params.update({param_name: i})
                return i

        def max_calories(self, max_calories):
            self.conditions.append("products.calories_100g <= %(max_calories)s")
            self.parameters.update({"max_calories": max_calories})
            return self

        def max_sugar(self, max_sugar):
            self.conditions.append("products.sugars_100g <= %(max_sugar)s")
            self.parameters.update({"max_sugar": max_sugar})
            return self

        def ingredient(self, ingredient):
            i = str(self.__param_index("ingredient"))
            self.joins.append(
                "JOIN product_ingredient as product_ingredient" + i + " on products.code = product_ingredient" + i + ".product JOIN ingredients as ingredients" + i + " on ingredients" + i + ".name = product_ingredient" + i + ".ingredient")
            self.conditions.append("LOWER(ingredients" + i + ".name) like LOWER(%(ingredient" + i + ")s)")
            self.parameters.update({"ingredient" + i: "%" + ingredient + "%"})
            return self

        def brand(self, brand):
            i = str(self.__param_index("brand"))
            self.conditions.append("LOWER(brands) like LOWER(%(brand" + i + ")s)")
            self.parameters.update({"brand" + i: "%" + brand + "%"})
            return self

        def category(self, category):
            i = str(self.__param_index("category"))
            self.joins.append(
                "JOIN product_category as product_category" + i + " on products.code = product_category" + i + ".product JOIN categories as categories" + i + " on categories" + i + ".name = product_category" + i + ".category")
            self.conditions.append("LOWER(categories" + i + ".name) like LOWER(%(category" + i + ")s)")
            self.parameters.update({"category" + i: "%" + category + "%"})
            return self

        def name(self, name):
            self.conditions.append("LOWER(products.name) like LOWER(%(name)s)")
            self.parameters.update({"name": name})
            return self
        
        def searchByCounty(self, name):
            i = str(self.__param_index("county"))
            
            self.conditions.append(""" "county" like %(county""" + i + ")s")
            #print(i)
            #print(name)
            #self.conditions.append(""" "COUNTY" like '%%""" + name + """%%' """)
            #print(self.conditions)
            self.parameters.update({"county" + i: "%" + name + "%"})
            #print(self.parameters)
            return self

        def searchByOperatorID(self, health_opID):
            i = str(self.__param_index("health_operation_id"))
            self.conditions.append(""" "nys_health_operation_id" = %(health_operation_id""" + i + ")s")
            #print(i)
            #print(name)
            #self.conditions.append(""" "COUNTY" like '%%""" + name + """%%' """)
            #print(self.conditions)
            self.parameters.update({"health_operation_id" + i: health_opID})
            #print(self.parameters)
            return self
        
        def searchByName(self, name):
            #query = """ select * from food_service_inspections JOIN food_service_operator_distinct on food_service_inspections."NYS HEALTH OPERATION ID" = food_service_operator_distinct."NYS HEALTH OPERATION ID" where "COUNTY" like '%%ALB%%' """
            i = str(self.__param_index("name"))
            self.joins.append(
                "JOIN food_service_operator as food_operator" + i + """ on food_service_inspections."nys_health_operation_id" = food_operator""" + i + """."nys_health_operation_id" """)
            self.conditions.append("food_operator" + i + """."operation_name" like %(name""" + i + ")s")
            self.parameters.update({"name" + i: "%" + name + "%"})
            return self
        '''
            print(query)
            with self.product_data.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
                #print(query)
                cursor.execute(query, self.parameters)
                return pandas.DataFrame(cursor.fetchall())
            return self
        '''
        def getAllViolations(self):
            query = """select * from food_service_violations order by violation_item"""
            with self.product_data.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
                #print(query)
                cursor.execute(query, self.parameters)
                return pandas.DataFrame(cursor.fetchall())
            return self

        def __compile(self):
            query = "SELECT DISTINCT * from food_service_inspections"
            query = query + " " + " ".join(self.joins) if self.joins else query
            query = query + " WHERE " + " AND ".join(self.conditions) if self.conditions else query
            #query += " ORDER BY food_service_inspections.code"
            return query

        def run(self):
            query = self.__compile()
            #query = """select * from food_service_inspections where "COUNTY" like 'ALB%%' limit 3"""
            with self.product_data.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
                print(query)
                cursor.execute(query, self.parameters)
                return pandas.DataFrame(cursor.fetchall())

        def show(self):
            query = self.__compile()
            with self.product_data.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
                return cursor.mogrify(query, self.parameters)

    def query(self):
        return self.FoodViolationQuery(self)

In [2]:
import re
import pandas
import matplotlib

In [3]:
fd_vio = FoodViolationData("host='localhost' dbname='test5' user='test5' password='test5'")

In [14]:
# Looking for estabishments that have the word BUFFET in their name
fd_vio.query().searchByName("BUFFET").run()

SELECT DISTINCT * from food_service_inspections JOIN food_service_operator as food_operator0 on food_service_inspections."nys_health_operation_id" = food_operator0."nys_health_operation_id"  WHERE food_operator0."operation_name" like %(name0)s


,county,date_of_inspection,nys_health_operation_id,violation_item,critical_violation,total_critical_violations,total_crit_not_corrected,total_noncritical_violations,nysdoh_gazetteer_1980,inspection_type,...,facility_city,facility_postal_zipcode,fs_facility_state,permitted_dba,permitted_corp_name,perm_operator_last_name,perm_operator_first_name,food_service_type,food_service_description,permit_expiration_date
0,ALBANY,2015-12-01,847550,10B,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
1,ALBANY,2015-12-01,847550,11D,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
2,ALBANY,2015-12-01,847550,14A,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
3,ALBANY,2015-12-01,847550,14B,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
4,ALBANY,2015-12-01,847550,15A,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
5,ALBANY,2015-12-01,847550,15B,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
6,ALBANY,2015-12-01,847550,8A,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
7,ALBANY,2015-12-01,847552,None,Not Critical Violation,0,0,0,15100,Inspection,...,DELMAR,12054,NY,None,None,None,None,Food Service Establishment,Frozen Desserts,2020-09-30
8,ALBANY,2016-08-31,847550,14B,Not Critical Violation,0,0,9,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
9,ALBANY,2016-08-31,847550,14C,Not Critical Violation,0,0,9,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30


In [15]:
# Looking for estabishments that have the word BUFFET in their name and are in ALBANY
fd_vio.query().searchByName("BUFFET").searchByCounty("ALBANY").run()

SELECT DISTINCT * from food_service_inspections JOIN food_service_operator as food_operator0 on food_service_inspections."nys_health_operation_id" = food_operator0."nys_health_operation_id"  WHERE food_operator0."operation_name" like %(name0)s AND  "county" like %(county0)s


,county,date_of_inspection,nys_health_operation_id,violation_item,critical_violation,total_critical_violations,total_crit_not_corrected,total_noncritical_violations,nysdoh_gazetteer_1980,inspection_type,...,facility_city,facility_postal_zipcode,fs_facility_state,permitted_dba,permitted_corp_name,perm_operator_last_name,perm_operator_first_name,food_service_type,food_service_description,permit_expiration_date
0,ALBANY,2015-12-01,847550,10B,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
1,ALBANY,2015-12-01,847550,11D,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
2,ALBANY,2015-12-01,847550,14A,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
3,ALBANY,2015-12-01,847550,14B,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
4,ALBANY,2015-12-01,847550,15A,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
5,ALBANY,2015-12-01,847550,15B,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
6,ALBANY,2015-12-01,847550,8A,Not Critical Violation,0,0,8,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
7,ALBANY,2015-12-01,847552,None,Not Critical Violation,0,0,0,15100,Inspection,...,DELMAR,12054,NY,None,None,None,None,Food Service Establishment,Frozen Desserts,2020-09-30
8,ALBANY,2016-08-31,847550,14B,Not Critical Violation,0,0,9,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30
9,ALBANY,2016-08-31,847550,14C,Not Critical Violation,0,0,9,15100,Inspection,...,DELMAR,12054,NY,None,"SISTER CHINESE BUFFET, INC.",ZHENG,HUI WEN,Food Service Establishment,Restaurant,2020-09-30


In [16]:
# Looking for estabishments that have the word BUFFET and the word DRAGON in their name and are in ALBANY
fd_vio.query().searchByName("BUFFET").searchByName("DRAGON").searchByCounty("ALBANY").run()

SELECT DISTINCT * from food_service_inspections JOIN food_service_operator as food_operator0 on food_service_inspections."nys_health_operation_id" = food_operator0."nys_health_operation_id"  JOIN food_service_operator as food_operator1 on food_service_inspections."nys_health_operation_id" = food_operator1."nys_health_operation_id"  WHERE food_operator0."operation_name" like %(name0)s AND food_operator1."operation_name" like %(name1)s AND  "county" like %(county0)s


,county,date_of_inspection,nys_health_operation_id,violation_item,critical_violation,total_critical_violations,total_crit_not_corrected,total_noncritical_violations,nysdoh_gazetteer_1980,inspection_type,...,facility_city,facility_postal_zipcode,fs_facility_state,permitted_dba,permitted_corp_name,perm_operator_last_name,perm_operator_first_name,food_service_type,food_service_description,permit_expiration_date
0,ALBANY,2019-08-15,1002730,10A,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
1,ALBANY,2019-08-15,1002730,11B,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
2,ALBANY,2019-08-15,1002730,11D,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
3,ALBANY,2019-08-15,1002730,12A,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
4,ALBANY,2019-08-15,1002730,12D,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
5,ALBANY,2019-08-15,1002730,12E,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
6,ALBANY,2019-08-15,1002730,14A,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
7,ALBANY,2019-08-15,1002730,14B,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
8,ALBANY,2019-08-15,1002730,15B,Not Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31
9,ALBANY,2019-08-15,1002730,5A,Critical Violation,4,0,17,15300,Inspection,...,ALBANY,12205,NY,None,"DRAGON BUFFET LI, INC.",LI,JING JING,Food Service Establishment,Restaurant,2020-03-31


In [107]:
fd_vio.query().getAllViolations()

,violation_item,violation_description
0,10A,Food (ice) contact surfaces are improperly des...
1,10B,Non-food contact surfaces and equipment are im...
2,11,"Thermometers inaccurate, improperly used, impr..."
3,11A,"Manual facilities inadequate, technique incorr..."
4,11B,"Wiping cloths dirty, not stored properly in sa..."
5,11C,"Food contact surfaces not washed, rinsed and s..."
6,11D,Non food contact surfaces of equipment not clean
7,12,"Personnel with unclean hands or clothing, toba..."
8,12A,"Hot, cold running water not provided, pressure..."
9,12B,Improperly functioning on-site sewage disposal...


SELECT * from food_service_inspections JOIN food_service_operator_distinct as food_operator0 on food_service_inspections."NYS HEALTH OPERATION ID" = food_operator0."NYS HEALTH OPERATION ID WHERE WHERE food_operator0."OPERATION NAME" like %(name0)s


SyntaxError: syntax error at or near "OPERATION"
LINE 1: ...S HEALTH OPERATION ID WHERE WHERE food_operator0."OPERATION ...
                                                             ^


In [79]:
# Just searching for "BANY" in COUNTY names
fd_vio.query().searchByCounty("BANY").run()

SELECT * from food_service_inspections WHERE  "COUNTY" like %(county0)s AND  "COUNTY" like '%%BANY%%' 


,COUNTY,DATE OF INSPECTION,NYS HEALTH OPERATION ID,VIOLATION ITEM,CRITICAL VIOLATION,TOTAL # CRITICAL VIOLATIONS,TOTAL #CRIT. NOT CORRECTED,TOTAL # NONCRITICAL VIOLATIONS,NYSDOH GAZETTEER 1980,INSPECTION TYPE,INSPECTION COMMENTS
0,ALBANY,2/11/2009,687080,12E,Not Critical Violation,0.0,0.0,3.0,10300,Inspection,None
1,ALBANY,2/3/2014,692526,8A,Not Critical Violation,1.0,0.0,6.0,15300,Inspection,None
2,ALBANY,8/23/2017,692526,14A,Not Critical Violation,0.0,0.0,3.0,15300,Inspection,excellent 8/23/2017
3,ALBANY,5/24/2010,676591,8A,Not Critical Violation,2.0,0.0,5.0,15500,Inspection,None
4,ALBANY,6/21/2017,676591,8E,Not Critical Violation,0.0,0.0,6.0,15500,Inspection,None
5,ALBANY,2/9/2011,676591,8E,Not Critical Violation,0.0,0.0,3.0,15500,Inspection,None
6,ALBANY,9/10/2009,676591,9B,Not Critical Violation,1.0,0.0,7.0,15500,Inspection,None
7,ALBANY,4/2/2012,676591,11A,Not Critical Violation,3.0,0.0,8.0,15500,Inspection,None
8,ALBANY,2/23/2015,676591,15A,Not Critical Violation,2.0,0.0,6.0,15500,Inspection,None
9,ALBANY,4/23/2019,676591,6A,Critical Violation,1.0,0.0,2.0,15500,Inspection,"Excellent compliance posted April 23, 2019\n\n..."


In [96]:
#Searching 3 at time, should return 0 rows
fd_vio.query().searchByCounty("BANY").searchByCounty("NAS").searchByCounty("Test").run()

SELECT * from food_service_inspections WHERE  "COUNTY" like %(county0)s AND  "COUNTY" like %(county1)s AND  "COUNTY" like %(county2)s


""
